In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Citibike Data") \
    .getOrCreate()


In [2]:
from pyspark.sql.types import StructType, StructField, StringType, DecimalType, TimestampType
from pyspark.sql.functions import create_map, lit

In [12]:
# 1️⃣ Define the mock class
class MockDBUtils:
    class Widgets:
        def __init__(self):
            self.values = {}

        def text(self, name, default_value=""):
            self.values[name] = default_value

        def get(self, name):
            return self.values.get(name, "")

    def __init__(self):
        self.widgets = self.Widgets()  # make widgets an attribute

# 2️⃣ Instantiate mock
dbutils = MockDBUtils()


In [13]:
pipeline_id = dbutils.widgets.get("pipeline_id")
run_id = dbutils.widgets.get("run_id")
task_id = dbutils.widgets.get("task_id")
processed_timestamp = dbutils.widgets.get("processed_timestamp")
catalog = dbutils.widgets.get("catalog")

In [19]:
dbutils = MockDBUtils()
dbutils.widgets.text("catalog","citibike_dev")
catalog = dbutils.widgets.get("catalog")
print("Using catalog:", catalog)

Using catalog: citibike_dev


In [15]:
schema = StructType([
    StructField("ride_id", StringType(), True),
    StructField("rideable_type", StringType(), True),
    StructField("started_at", TimestampType(), True),
    StructField("ended_at", TimestampType(), True),
    StructField("start_station_name", StringType(), True), 
    StructField("start_station_id", StringType(), True),   
    StructField("end_station_name", StringType(), True), 
    StructField("end_station_id", StringType(), True), 
    StructField("start_lat", DecimalType(), True), 
    StructField("start_lng", DecimalType(), True), 
    StructField("end_lat", DecimalType(), True), 
    StructField("end_lng", DecimalType(), True), 
    StructField("member_casual", StringType(), True), 
])

In [16]:
df = spark.read.csv(f"/Volumes/{catalog}/00_landing/source_citibike_data/JC-202503-citibike-tripdata.csv", schema=schema, header=True)

In [17]:
df = df.withColumn("metadata", 
              create_map(
                  lit("pipeline_id"), lit(pipeline_id),
                  lit("run_id"), lit(run_id),
                  lit("task_id"), lit(task_id),
                  lit("processed_timestamp"), lit(processed_timestamp)
                  ))

In [ ]:
df.write.\
    mode("overwrite").\
    option("overwriteSchema", "true").\
    saveAsTable(f"{catalog}.01_bronze.jc_citibike")